# Machine Learning in R: part 2

## This week's tutorial focuses on improvement. We will use some tuning methods to improve the predictive ability of the models we build.


## 1a. Cleaning and formatting the data (from last week)

Below is the code from the loading, cleaning and train/test split sections we went over last week. This is all we requre to get the data into the format needed to being training some machine learning models.

In [2]:
library(tidyverse)
library(xgboost)

housing = read.csv('./housing.csv')

housing$total_bedrooms[is.na(housing$total_bedrooms)] = median(housing$total_bedrooms , na.rm = TRUE)

housing$mean_bedrooms = housing$total_bedrooms/housing$households
housing$mean_rooms = housing$total_rooms/housing$households

drops = c('total_bedrooms', 'total_rooms')

housing = housing[ , !(names(housing) %in% drops)]

categories = unique(housing$ocean_proximity)
#split the categories off
cat_housing = data.frame(ocean_proximity = housing$ocean_proximity)

for(cat in categories){
    cat_housing[,cat] = rep(0, times= nrow(cat_housing))
}

for(i in 1:length(cat_housing$ocean_proximity)){
    cat = as.character(cat_housing$ocean_proximity[i])
    cat_housing[,cat][i] = 1
}

cat_columns = names(cat_housing)
keep_columns = cat_columns[cat_columns != 'ocean_proximity']
cat_housing = select(cat_housing,one_of(keep_columns))
drops = c('ocean_proximity','median_house_value')
housing_num =  housing[ , !(names(housing) %in% drops)]


scaled_housing_num = scale(housing_num)

cleaned_housing = cbind(cat_housing, scaled_housing_num, median_house_value=housing$median_house_value)


set.seed(19) # Set a random seed so that same sample can be reproduced in future runs

sample = sample.int(n = nrow(cleaned_housing), size = floor(.8*nrow(cleaned_housing)), replace = F)
train = cleaned_housing[sample, ] #just the samples
test  = cleaned_housing[-sample, ] #everything but the samples


train_y = train[,'median_house_value']
train_x = train[, names(train) !='median_house_value']

test_y = test[,'median_house_value']
test_x = test[, names(test) !='median_house_value']

head(train)

,NEAR BAY,<1H OCEAN,INLAND,NEAR OCEAN,ISLAND,longitude,latitude,housing_median_age,population,households,median_income,mean_bedrooms,mean_rooms,median_house_value
2418,0,0,1,0,0,0.06473791,0.4485767,-0.05081113,-0.08342596,-0.50882695,-1.2394168,-0.03648780,-0.4145713,56700
9990,0,0,1,0,0,-0.74882545,1.6471053,-1.08374113,1.39212008,2.14071836,-0.7358959,-0.19291092,-0.1004065,143400
13440,0,0,1,0,0,1.07295753,-0.7218613,-0.05081113,0.28656434,0.06136148,0.1404495,-0.18700644,0.2732884,128300
1412,1,0,0,0,0,-1.25293526,1.0759315,0.50538194,0.35897294,0.42492199,0.6344959,-0.11581168,0.2741324,233200
7539,0,1,0,0,0,0.67865382,-0.8061329,-0.20972344,1.03802435,0.21829408,-1.0991931,-0.03247975,-0.5151724,110200
4621,0,1,0,0,0,0.62874196,-0.7265431,1.61776810,0.10024464,0.24706505,-0.6573622,-0.07763347,-0.4598522,350900


## 1b. Cleaning - The tidyverse way! 

The code below does the same thing as the code above, but employs the tidyverse. I've pulled out the bare bones parts of Karl's 'Housing_R_tidy.r' script needed to get the data to where we want it (i.e. removed all the graphs head commands etc. Go to his original script to see the notes and visual blandishments).

I like this code more then my original version because:
1. It is easy to follow the workflow. magrittr makes it easy to see when one cleaning task ends and the next begins.
2. It is more concise.
3. The use of comments(#) after the pipes(%>%) looks professional and also makes the code more readable. Being able to share your code with others and have them understand it is very important!
4. It runs faster.

Note: in the tibble docs the function is listed as: as_tibble() not as.tibble() as first written. Oddly as.tibble() worked in my normal R deployment, but threw an error in the jupyter notebook :\ This confused me and I don't know what to make of it.

In [ ]:
library(tidyverse)

housing.tidy = read_csv('housing.csv')

housing.tidy = housing.tidy %>% 
  mutate(total_bedrooms = ifelse(is.na(total_bedrooms), 
                                 median(total_bedrooms, na.rm = T),
                                 total_bedrooms),
         mean_bedrooms = total_bedrooms/households,
         mean_rooms = total_rooms/households) %>%
  select(-c(total_rooms, total_bedrooms))


categories = unique(housing.tidy$ocean_proximity) # all categories

cat_housing.tidy = categories %>% # compare the full vector against each category consecutively
  lapply(function(x) as.numeric(housing.tidy$ocean_proximity == x)) %>% # convert to numeric
  do.call("cbind", .) %>% as_tibble() # clean up
colnames(cat_housing.tidy) = categories # make nice column names

cleaned_housing.tidy = housing.tidy %>% 
  select(-c(ocean_proximity, median_house_value)) %>%
  scale() %>% as_tibble() %>%
  bind_cols(cat_housing.tidy) %>%
  add_column(median_house_value = housing.tidy$median_house_value)

set.seed(19) # Set a random seed so that same sample can be reproduced in future runs

sample = sample.int(n = nrow(cleaned_housing.tidy), size = floor(.8*nrow(cleaned_housing.tidy)), replace = F)
train = cleaned_housing.tidy[sample, ] #just the samples
test  = cleaned_housing.tidy[-sample, ] #everything but the samples
      
head(train)


## 2a. Last week's random forest model

This is the model we made at the end of last week's session I'm running it again here so that we have a benchmark to for the other models we train today.

In [2]:

########
# Random Forest Model
########
library(randomForest)
rf_model = randomForest(train_x, y = train_y , ntree = 500, importance = TRUE)

names(rf_model) #these are all the different things you can call from the model.

importance_dat = rf_model$importance
importance_dat

sorted_predictors = sort(importance_dat[,1], decreasing=TRUE)
sorted_predictors

oob_prediction = predict(rf_model) #leaving out a data source forces OOB predictions

#you may have noticed that this is avaliable using the $mse in the model options.
#but this way we learn stuff!
train_mse = mean(as.numeric((oob_prediction - train_y)^2))
oob_rmse = sqrt(train_mse)
oob_rmse


y_pred_rf = predict(rf_model , test_x)
test_mse = mean(((y_pred_rf - test_y)^2))
test_rmse = sqrt(test_mse)
test_rmse # ~48620

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



[1] "call"            "type"            "predicted"       "mse"            
 [5] "rsq"             "oob.times"       "importance"      "importanceSD"   
 [9] "localImportance" "proximity"       "ntree"           "mtry"           
[13] "forest"          "coefs"           "y"               "test"           
[17] "inbag"

,%IncMSE,IncNodePurity
NEAR BAY,4.450369e+08,1.391014e+12
<1H OCEAN,1.427100e+09,4.338351e+12
INLAND,3.703060e+09,3.116839e+13
NEAR OCEAN,4.262002e+08,2.119833e+12
ISLAND,6.761432e+04,1.682004e+10
longitude,6.686360e+09,2.580455e+13
latitude,5.375308e+09,2.235183e+13
housing_median_age,1.062666e+09,9.808021e+12
population,1.081431e+09,7.441795e+12
households,1.193537e+09,7.897400e+12


median_income          longitude           latitude             INLAND 
      8.325735e+09       6.686360e+09       5.375308e+09       3.703060e+09 
        mean_rooms          <1H OCEAN         households         population 
      1.890830e+09       1.427100e+09       1.193537e+09       1.081431e+09 
housing_median_age           NEAR BAY         NEAR OCEAN      mean_bedrooms 
      1.062666e+09       4.450369e+08       4.262002e+08       4.083603e+08 
            ISLAND 
      6.761432e+04

[1] 48973.79

[1] 48392.87

$48392 is the test error benchmark based off that random forest run.

## 2b. Gradient Boosting

Gradient boosting is an ensemble supervised machine learning model that builds up the concept of the random forest algorithm we explored last week. Recall that for a random forest we spawned 500 decision trees and took the mean of their predictions to get a 'wisdom of the crowd' effect and arrive at a more accurate prediction than any one tree would provide.

Here we use the Extreme Gradient Boosting library to implement this in R. Note the 'Extreme' in extreme gradient boosting refers to the computational efficiency. The algorithm could more accurately could be described as 'regularized gradient boosting'.

###  Gradient Boosting - Algorithm details

Extreme gradient boosting also builds a forest of trees, but does so in an additive manner. The algorithm iteratively builds trees that minimize the error, and thereby descends towards an optimal set of predictive trees. Already learned trees are kept, and new trees are added one after another to minimize the objective function (error in predictions). The trees are grown sequentially: each tree is grown using information from previously grown trees. Each tree is fit on a modified version of the original data set based on the previous trees built.

The trees are accompanied by a regularization paramater to avoid overfit.  

One difference between boosting and random forests: in boosting, because the growth of a particular 
tree takes into account the other trees that have already been grown, smaller trees are typically sufficient (less splits and depth). 

In [3]:
######
# XG Boost
######
# see the docs: http://cran.fhcrc.org/web/packages/xgboost/vignettes/xgboost.pdf
library(xgboost)

#put into the xgb matrix format
dtrain = xgb.DMatrix(data =  as.matrix(train_x), label = train_y )
dtest = xgb.DMatrix(data =  as.matrix(test_x), label = test_y)

# these are the datasets the rmse is evaluated for at each iteration
watchlist = list(train=dtrain, test=dtest)

# try 1 - off a set of paramaters I know work pretty well for most stuff

bst = xgb.train(data = dtrain, 
                max.depth = 8, 
                eta = 0.3, 
                nthread = 2, 
                nround = 1000, 
                watchlist = watchlist, 
                objective = "reg:linear", 
                early_stopping_rounds = 50,
                print_every_n = 50)

[1]	train-rmse:171405.234375	test-rmse:173186.609375 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 50 rounds.

[51]	train-rmse:22239.695312	test-rmse:47789.132812 
[101]	train-rmse:15978.763672	test-rmse:47781.835938 
Stopping. Best iteration:
[56]	train-rmse:21571.712891	test-rmse:47723.835938



So our first run there gets a rmse of $47723, an improvement over our benchmark model. That isn't the end of the story though, we can try to squeak out further improvements through 'hyperparameter tuning'. A hyperparameter is one of the mutable options that we pass to the algorithm along with our data.

## 2c. Tuning the algorithm - hyperparameters for xgboost

Boosting has 3 tuning paramaters that we can focus on

1. The number of trees. Here we use a good trick, instead of specifying an exact number, we give the algorithm a big number (nround = 10000) and the param (early_stopping_rounds = 50). This effectively means: 'keep iteratively growing trees until you have 10,000 of them, or stop early if the scores haven't improved for the last 50 rounds'.
2. The shrinkage parameter λ (eta in the params), a small positive number. This controls the rate at which boosting learns. Typical values are 0.01 or 0.001, and the right choice can depend on the problem. Very small λ can require using a very large value of B in order to achieve good performance.
3. The number of splits in each tree, which controls the complexity of the boosted ensemble (controlled with max.depth).

### Here we try a 'slower learning' model. The up and down weights for each iteration are smaller we also use more iterations to account for the fact that the model will take longer to learn.

In [4]:
bst_slow = xgb.train(data = dtrain, 
                        max.depth=5, 
                        eta = 0.01, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:linear", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

[1]	train-rmse:234537.453125	test-rmse:235377.093750 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 50 rounds.

[501]	train-rmse:49887.167969	test-rmse:53810.851562 
[1001]	train-rmse:43695.054688	test-rmse:49724.628906 
[1501]	train-rmse:39971.750000	test-rmse:48036.140625 
[2001]	train-rmse:37479.328125	test-rmse:47158.046875 
[2501]	train-rmse:35481.843750	test-rmse:46556.929688 
[3001]	train-rmse:33934.000000	test-rmse:46190.351562 
[3501]	train-rmse:32535.261719	test-rmse:45904.425781 
[4001]	train-rmse:31228.515625	test-rmse:45617.722656 
[4501]	train-rmse:30171.023438	test-rmse:45451.519531 
[5001]	train-rmse:29281.529297	test-rmse:45383.992188 
[5501]	train-rmse:28471.605469	test-rmse:45289.558594 
[6001]	train-rmse:27689.521484	test-rmse:45230.863281 
Stopping. Best iteration:
[5959]	train-rmse:27752.142578	test-rmse:45225.968750



In [5]:
rf_benchmark = 48392

bst_slow$best_score / rf_benchmark

test-rmse 
0.9345753

Things to note:
- 6009 iterations were run, and it backtracked to 5959 to get the best one thanks to our early stopping rounds parameter.
- So that is an improvement of ~6.5% in rmse error over last week. Yay lets go home. Wait! What we have done here is fit to the training set and the test set at the same time (which can, and likelt has, lead to model overfit).  

## Problems to address

### 1. We need to work with a validation set and only at the end evaluate the model performance against the test set.
Remember our test set should be withheld to evaluate the model on data it hasn't seen. By iteratively checking the train and test rmse above, we have violated that rule and so we cannot say how accurate our model is on data it hasn't seen. To do this we need a validation set (essentially a second test set) that the model can peek at after each iteration to see how well it is performing or external data, then with the final version we can make the assessment vs. the test set.

### 2 .If we kept tweaking one hyperparameter, waiting to see the result and then repeating the process we will be here forever. We need to speed this up in a systematic fashion


## 2d. A validation set

validation set - Another subset of our data that is witheld from the training algorithm, but compared against at each iteration to see how the model is performing.

Here we make this through the same method as the test set, by sampling 20% of the remaining training set and passing this into the xgboost watchlist. The algorithm will watch the rmse of the training (which it can learn from) and the validation (which it can't learn parameters from, only see the rmse outcome) and continue until one is no longer improving


### Note on tidyverse - I you switch the first cell to Karl's tidyverse version, then you need to make the following switch
Karl's notebook: https://www.kaggle.com/karlcottenie/introduction-to-machine-learning-in-r-tutorial

train_y = train_t[,'median_house_value']
When I run the tidy cleaning for step one this outputs a dataframe like column, but the algorithm needs a plain vector of numbers for the y labels.

The code needs to switch to:

train_y = pull(train_t, median_house_value)

In order to un-tibble the data

In [6]:
####
# Proper use - validation set
####


sample = sample.int(n = nrow(train), size = floor(.8*nrow(train)), replace = F)

train_t = train[sample, ] #just the samples
valid  = train[-sample, ] #everything but the samples

train_y = train_t[,'median_house_value']

#if tidyverse was used, dplyr pull function solves the problem:
#train_y = pull(train_t, median_house_value)
train_x = train_t[, names(train_t) !='median_house_value']

valid_y = valid[,'median_house_value']
valid_x = valid[, names(train_t) !='median_house_value']

train_y[1:10]

[1] 476400 409900 235500  74700 171100 188600  94400 211100 450000 228100

In [7]:
gb_train = xgb.DMatrix(data = as.matrix(train_x), label = train_y )
gb_valid = xgb.DMatrix(data = as.matrix(valid_x), label = valid_y )
#in jupyter the label needs to be in an as.matrix() or I get an error? subtle and annoying differences

# train xgb, evaluating against the validation
watchlist = list(train = gb_train, valid = gb_valid)


We then run the xgboost algorithm again and after training we evaluate on the test data.

In [8]:
bst_slow = xgb.train(data= gb_train, 
                        max.depth = 10, 
                        eta = 0.01, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:linear", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

[1]	train-rmse:234965.703125	valid-rmse:232542.093750 
Multiple eval metrics are present. Will use valid_rmse for early stopping.
Will train until valid_rmse hasn't improved in 50 rounds.

[501]	train-rmse:23408.917969	valid-rmse:48958.464844 
[1001]	train-rmse:15327.182617	valid-rmse:47956.554688 
[1501]	train-rmse:11868.001953	valid-rmse:47733.187500 
[2001]	train-rmse:9716.176758	valid-rmse:47619.566406 
[2501]	train-rmse:8043.468750	valid-rmse:47542.125000 
Stopping. Best iteration:
[2626]	train-rmse:7657.586914	valid-rmse:47534.148438



In [9]:

# recall we ran the following to get the test data in the right format:
# dtest = xgb.DMatrix(data =  as.matrix(test_x), label = test_y)
# here I have it with the label taken off, just to remind us its external data xgb would ignore the label though during predictions
dtest = xgb.DMatrix(data =  as.matrix(test_x))

#test the model on truly external data

y_hat_valid = predict(bst_slow, dtest)

test_mse = mean(((y_hat_valid - test_y)^2))
test_rmse = sqrt(test_mse)
test_rmse 


[1] 47331.16

In [10]:
test_rmse/rf_benchmark

[1] 0.9780782

This is higher then on the first run, but we can be confident that the improved score is not due to overfit thanks to our use of a validation set! A lower rmse isn't necessarily better if it comes at the cose of overfit, we now have more confidence in external predictions.


## 3a. Grid Search to find the best hyperparameter combinations

In [11]:

###
# Grid search first principles 
###

max.depths = c(7, 9)
etas = c(0.01, 0.001)

best_params = 0
best_score = 0

count = 1
for( depth in max.depths ){
    for( num in etas){

        bst_grid = xgb.train(data = gb_train, 
                                max.depth = depth, 
                                eta=num, 
                                nthread = 2, 
                                nround = 10000, 
                                watchlist = watchlist, 
                                objective = "reg:linear", 
                                early_stopping_rounds = 50, 
                                verbose=0)

        if(count == 1){
            best_params = bst_grid$params
            best_score = bst_grid$best_score
            count = count + 1
            }
        else if( bst_grid$best_score < best_score){
            best_params = bst_grid$params
            best_score = bst_grid$best_score
        }
    }
}

best_params
best_score





$max_depth
[1] 7

$eta
[1] 0.01

$nthread
[1] 2

$objective
[1] "reg:linear"

$silent
[1] 1

valid-rmse 
  47117.84

In [12]:
# max_depth of 7, eta of 0.01
bst_tuned = xgb.train( data = gb_train, 
                        max.depth = 7, 
                        eta = 0.01, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:linear", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

y_hat_xgb_grid = predict(bst_tuned, dtest)

test_mse = mean(((y_hat_xgb_grid - test_y)^2))
test_rmse = sqrt(test_mse)
test_rmse # test-rmse: 46675



[1]	train-rmse:234989.328125	valid-rmse:232543.312500 
Multiple eval metrics are present. Will use valid_rmse for early stopping.
Will train until valid_rmse hasn't improved in 50 rounds.

[501]	train-rmse:38440.367188	valid-rmse:50656.929688 
[1001]	train-rmse:31137.027344	valid-rmse:48560.308594 
[1501]	train-rmse:26716.404297	valid-rmse:47650.949219 
[2001]	train-rmse:23859.535156	valid-rmse:47432.820312 
[2501]	train-rmse:21716.740234	valid-rmse:47233.796875 
[3001]	train-rmse:19926.523438	valid-rmse:47135.058594 
Stopping. Best iteration:
[3109]	train-rmse:19554.800781	valid-rmse:47117.835938



[1] 46146.48

In [13]:
test_rmse/rf_benchmark

[1] 0.9535972

By tuning the hyperparamaters we have made a slightly greater improvement over random forest. It is however only a small improvement over the non tuned xgboost model. But these performance differences do matter in some circumstances!


## 3b. Efficiently tweak the hyperparamaters using a grid search/cross-validation

The caret package (short for classification and regression training) is used to simplify the grid search we just implemented. We can just pass it a grid of hyperparameter combinations and it will run all the combinations and do a cross-validation for each (so no validation set needed)

[caret info](http://topepo.github.io/caret/index.html)

Similar to the tidyverse it works really well... but you have to learn all the code tricks!


In [14]:
library(caret) 

# look up the model we are running to see the paramaters
modelLookup("xgbLinear")
 
# set up all the pairwise combinations

xgb_grid_1 = expand.grid(nrounds = c(1000,2000,3000,4000) ,
                            eta = c(0.01, 0.001, 0.0001),
                            lambda = 1,
                            alpha = 0)
xgb_grid_1


#here we do one better then a validation set, we use cross validation to 
#expand the amount of info we have!
xgb_trcontrol_1 = trainControl(method = "cv",
                                number = 5,
                                verboseIter = TRUE,
                                returnData = FALSE,
                                returnResamp = "all", 
                                allowParallel = TRUE)


Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift



model,parameter,label,forReg,forClass,probModel
xgbLinear,nrounds,# Boosting Iterations,TRUE,TRUE,TRUE
xgbLinear,lambda,L2 Regularization,TRUE,TRUE,TRUE
xgbLinear,alpha,L1 Regularization,TRUE,TRUE,TRUE
xgbLinear,eta,Learning Rate,TRUE,TRUE,TRUE


nrounds,eta,lambda,alpha
1000,1e-02,1,0
2000,1e-02,1,0
3000,1e-02,1,0
4000,1e-02,1,0
1000,1e-03,1,0
2000,1e-03,1,0
3000,1e-03,1,0
4000,1e-03,1,0
1000,1e-04,1,0
2000,1e-04,1,0


Train the model for each parameter combination in the grid, using CV to evaluate on multiple folds. Make sure your laptop is plugged in or else RIP battery if you've got a tiny old macbook like me.


In [15]:
######
#below a grid-search, cross-validation xgboost model in caret
######


xgb_train_1 = train(x = as.matrix(train_x),
                    y = train_y,
                    trControl = xgb_trcontrol_1,
                    tuneGrid = xgb_grid_1,
                    method = "xgbLinear",
                    max.depth = 5)

names(xgb_train_1)
xgb_train_1$bestTune
xgb_train_1$method
summary(xgb_train_1)


#alternatively, you can 'narrow in' on the best paramaters. Repeat the above by taking a range of options around 
#the best values found and seeing if high resolution tweaks can provide even further improvements.

xgb_cv_yhat = predict(xgb_train_1 , as.matrix(test_x))


test_mse = mean(((xgb_cv_yhat - test_y)^2))
test_rmse = sqrt(test_mse)
test_rmse # 47744... higher then 'by hand' grid search!



+ Fold1: nrounds=1000, eta=1e-02, lambda=1, alpha=0 
- Fold1: nrounds=1000, eta=1e-02, lambda=1, alpha=0 
+ Fold1: nrounds=2000, eta=1e-02, lambda=1, alpha=0 
- Fold1: nrounds=2000, eta=1e-02, lambda=1, alpha=0 
+ Fold1: nrounds=3000, eta=1e-02, lambda=1, alpha=0 
- Fold1: nrounds=3000, eta=1e-02, lambda=1, alpha=0 
+ Fold1: nrounds=4000, eta=1e-02, lambda=1, alpha=0 
- Fold1: nrounds=4000, eta=1e-02, lambda=1, alpha=0 
+ Fold1: nrounds=1000, eta=1e-03, lambda=1, alpha=0 
- Fold1: nrounds=1000, eta=1e-03, lambda=1, alpha=0 
+ Fold1: nrounds=2000, eta=1e-03, lambda=1, alpha=0 
- Fold1: nrounds=2000, eta=1e-03, lambda=1, alpha=0 
+ Fold1: nrounds=3000, eta=1e-03, lambda=1, alpha=0 
- Fold1: nrounds=3000, eta=1e-03, lambda=1, alpha=0 
+ Fold1: nrounds=4000, eta=1e-03, lambda=1, alpha=0 
- Fold1: nrounds=4000, eta=1e-03, lambda=1, alpha=0 
+ Fold1: nrounds=1000, eta=1e-04, lambda=1, alpha=0 
- Fold1: nrounds=1000, eta=1e-04, lambda=1, alpha=0 
+ Fold1: nrounds=2000, eta=1e-04, lambda=1, al

[1] "method"       "modelInfo"    "modelType"    "results"      "pred"        
 [6] "bestTune"     "call"         "dots"         "metric"       "control"     
[11] "finalModel"   "preProcess"   "trainingData" "resample"     "resampledCM" 
[16] "perfNames"    "maximize"     "yLimits"      "times"        "levels"

nrounds,lambda,alpha,eta
1000,1,0,1e-04


[1] "xgbLinear"

              Length  Class              Mode       
handle              1 xgb.Booster.handle externalptr
raw           2073968 -none-             raw        
niter               1 -none-             numeric    
call                6 -none-             call       
params              5 -none-             list       
callbacks           1 -none-             list       
feature_names      13 -none-             character  
xNames             13 -none-             character  
problemType         1 -none-             character  
tuneValue           4 data.frame         list       
obsLevels           1 -none-             logical    
param               1 -none-             list       

[1] 47744.17

Cam's hypothesis on caret performance - we are not using 'early stopping rounds' here so the model isn't cutting out at the exact best point. Re-running this with a validation setup as opposed to a cv setup would allow us to implement a grid search efficiently and wind up with the best hyperparamaters. Here we also didn't tweak tree depth, so that may in fact be important for the performance gains we saw. I shall leave this as a follow up exercise for the curious.


## 4 Ensemble the models together 

This is a good strategy for when accuracy is more important then knowing the best predictors.


In [16]:
#y_pred_rf #random forest
#y_hat_valid #xgBoost with validation
#y_hat_xgb_grid #xgBoost grid search
#xgb_cv_yhat #xgBoost caret cross validation

length(y_hat_xgb_grid)


blend_pred = (y_hat_valid * .25) + (y_pred_rf * .25) + (xgb_cv_yhat * .25) + (y_hat_xgb_grid * .25)
length(blend_pred)

length(blend_pred) == length(y_hat_xgb_grid)

blend_test_mse = mean(((blend_pred - test_y)^2))
blend_test_rmse = sqrt(blend_test_mse)
blend_test_rmse 


[1] 4128

[1] 4128

[1] TRUE

[1] 45407.74

Just by averaging  4 (very similar) predictors we have dropped the rmse a few percent lower then the best scoring of the 4 models. This does come at a cost though, we now can't make accurate inferences about the best predictors! The strategy is more effective when you take a more diverse set of models and ensemble those together.

### next step - you can grid search the weights of the ensemble to try and drop the rmse further!
